<a href="https://colab.research.google.com/github/LimGuenTaek/Object_Detection/blob/master/Feature_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%cd drive/MyDrive/SSD_tutorial/

/content/drive/MyDrive/SSD_tutorial


# Feature_Map Visualization

* checkpoint에 학습된 model이 담겨있는 checkpoint 건내주기
* 이미지는 PIL class로 받아오고(비교를 위해 , 성능이 차이가 났던 이미지를 사용하는게 좋을 것이라 생각이 듬)
* Feature_Map 저장을 어떻게 할지 고민중... 이름이 index로 저장해주니깐 헷갈림


In [3]:
import torch
from utils import *
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint = '/content/drive/MyDrive/SSD_tutorial/tar/SSD(2007+2012).tar'
checkpoint = torch.load(checkpoint)
model = checkpoint['model']
model = model.to(device)

In [4]:
import torchvision.transforms.functional as FT
image = Image.open('/content/drive/MyDrive/sample.jpg', mode='r') 
image = image.convert('RGB')

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
dims=(300, 300)

new_image = FT.resize(image, dims)

new_image = FT.to_tensor(new_image)

new_image = FT.normalize(new_image, mean=mean, std=std)

image=[]
image.append(new_image)
image=torch.stack(image,dim=0)
print(image.shape) # Tensor로 변환 성공 

torch.Size([1, 3, 300, 300])


* 이제 checkpoint로 train된 model을 불러와서 이미지 넣어주고 feature_map을 반환 받고 

* matplotlib로 visualization 행해주면 끝

In [5]:
model.eval()
image=image.to(device)

layer=[]
layer.append(model.base.conv1_1)
layer.append(model.base.conv1_2) 
layer.append(model.base.pool1)
layer.append(model.base.conv2_1)
layer.append(model.base.conv2_2) 
layer.append(model.base.pool2)
layer.append(model.base.conv3_1)
layer.append(model.base.conv3_2)
layer.append(model.base.conv3_3) 
layer.append(model.base.pool3)
layer.append(model.base.conv4_1)
layer.append(model.base.conv4_2)
layer.append(model.base.conv4_3) 
layer.append(model.base.pool4)
layer.append(model.base.conv5_1)
layer.append(model.base.conv5_2)
layer.append(model.base.conv5_3) 
layer.append(model.base.pool5)
layer.append(model.base.conv6)
layer.append(model.base.conv7)
layer.append(model.aux_convs.conv8_1)
layer.append(model.aux_convs.conv8_2)
layer.append(model.aux_convs.conv9_1)
layer.append(model.aux_convs.conv9_2)
layer.append(model.aux_convs.conv10_1)
layer.append(model.aux_convs.conv10_2)
layer.append(model.aux_convs.conv11_1)
layer.append(model.aux_convs.conv11_2)

# Feature-Map Extraction

* 우선 두 model에 대해 11개의 feature는 동일하게 뽑아보고 ,

* Fused 된 feature들 6개 또한 따로 뽑아본다.

In [6]:
import torch.nn.functional as F
from torch import nn

feature=[]
out=layer[0](image)
feature.append(out.to('cpu'))

for i in range(1,len(layer)):
  out=layer[i](out)
  if isinstance(layer[i],nn.Conv2d):
    feature.append(out.to('cpu'))


In [ ]:
import matplotlib.pyplot as plt # to display and save the filters and feature map images

# visualize 64 features from each layer , although there are more feature maps in the upper layers
for num_layer in range(len(feature)):
    plt.figure(figsize=(10, 10))
    layer_viz = feature[num_layer][0, :, :, :]
    layer_viz = layer_viz.data
    print(layer_viz.size())
    for i, filter in enumerate(layer_viz):
        if i == 1: # we will visualize only 8x8 blocks from each layer
            break
        plt.subplot(1, 1, i + 1)
        plt.imshow(filter, cmap='gray')
        plt.axis("off")
    print(f"Saving layer {num_layer} feature maps...")
    plt.savefig(f"/content/drive/MyDrive/Feature_map/Original/{num_layer}.png")
    plt.show()
    plt.close()